In [2]:
from argparse import ArgumentParser
import psutil as ps
from time import sleep, time
import uuid
import paho.mqtt.client as mqtt
import json
import redis

In [3]:
#Create redis client
REDIS_HOST = 'redis-11230.c293.eu-central-1-1.ec2.cloud.redislabs.com'
REDIS_PORT = 11230
REDIS_USER = 'default'
REDIS_PASSWORD = '2DXp9ovUfPWU8NTvfFaby0ZODJJRQnj8'

redis_client = redis.Redis(
    host = REDIS_HOST,
    port = REDIS_PORT,
    username = REDIS_USER,
    password = REDIS_PASSWORD
    )

# Encoder for the labels, since the value for redis takes only numbers
encoded_labels = {
    'car':1,
    'bus':2,
    'chatter':3,
    'motorcycle':4,
    'other':5,
    'truck':6
    }

#Check if it is connected
is_connected = redis_client.ping()
print(is_connected)


True


In [4]:
#Create the mqtt client
client =  mqtt.Client()
# redis_client.flushDb() # Da capire come si fa per cancellare Redis

In [5]:
# Topic is the city (montevideo)
# And the payload will contain the mac address of the zone (Magallanes, Acevedo, Rivera, Templete)

topic = 'montevideo'

# Callbacks
def on_connect(client, userdata, flags, rc): #triggered by connection
    # Subscribe for a topic
    client.subscribe(topic)

def on_message(client, userdata, msg): #triggered by an incoming message
    # Take the message payload and decode it since it is binary
    message = msg.payload.decode()
    #Get the message topic
    topic = msg.topic
    print(f'Message received :\n {message}\nFrom: {topic}')

    #Convert the msg payload into a dictionary
    message = json.loads(message)

    mac_address = message['mac_address']
    timestamp_in_ms = message['timestamp']
    label = message['label']

    #Get the timeseries names
    key = f"{mac_address}"
    encoded_label = encoded_labels[label]

    #If the timeseries does not exist,
    # create it
    if not redis_client.ts().execute_command(f'EXISTS {key}'):
        redis_client.ts().create(key, uncompressed=False, chunk_size=128) # Timeseries for label \in (0,1.0)
    #Store the data on redis
    redis_client.ts().add(key, timestamp=timestamp_in_ms, value=encoded_label)
    

In [6]:
#Bind the callbacks with the client
client.on_connect = on_connect
client.on_message = on_message

In [7]:
#Connect the client to the Message Broker
client.connect(
    host = 'mqtt.eclipseprojects.io',
    port = 1883
)

0

In [8]:
#Listen to infinity
client.loop_forever()

 {"mac_address": "0x19f57bb40d93", "timestamp": 1681393274269, "label": "motorcycle"}
From: montevideo
0x19f57bb40d93
Message received :
 {"mac_address": "0x19f57bb40d93", "timestamp": 1681393279266, "label": "car"}
From: montevideo
0x19f57bb40d93
Message received :
 {"mac_address": "0x19f57bb40d93", "timestamp": 1681393280264, "label": "car"}
From: montevideo
0x19f57bb40d93
Message received :
 {"mac_address": "0x19f57bb40d93", "timestamp": 1681393281265, "label": "motorcycle"}
From: montevideo
0x19f57bb40d93
Message received :
 {"mac_address": "0x19f57bb40d93", "timestamp": 1681393283262, "label": "motorcycle"}
From: montevideo
0x19f57bb40d93
Message received :
 {"mac_address": "0x19f57bb40d93", "timestamp": 1681393284263, "label": "car"}
From: montevideo
0x19f57bb40d93
Message received :
 {"mac_address": "0x19f57bb40d93", "timestamp": 1681393287262, "label": "motorcycle"}
From: montevideo
0x19f57bb40d93
Message received :
 {"mac_address": "0x19f57bb40d93", "timestamp": 1681393288260,

KeyboardInterrupt: 

### For debug purposes

In [0]:
mac_address = '0x71685e74f197'
mac_battery = f"{mac_address}:battery"
mac_power = f"{mac_address}:power"

print(mac_battery)

#Check if the data are stored
#print(f"Redis battery record: {redis_client.keys(match='0x([0-9a-fA-F]{12})')}")
print(f'Redis power record: {redis_client.ts().get(mac_power)}')

In [0]:
#redis_client.flushdb()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=251b70cc-64ed-4269-9639-75a0d233eb14' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>